### Import Required Libraries and Set Up Environment Variables

In [135]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
from pandas import json_normalize
import os
import pandas as pd
import json

In [136]:
# Set environment variables from the .env in the local environment
load_dotenv("example.env")
nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"
query_url

'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=OyNcyr1RskB6KmoVpVzSz7AeZ7JwNy2z&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531'

In [142]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    query = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query)
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    reviews = response.json()
    try:
    # Try and save the reviews to the reviews_list
    # loop through the reviews["response"]["docs"] and append each review to the list
        for doc in reviews["response"]["docs"]:
            reviews_list.append(doc)
      
        # Print the page that was just retrieved
        print(f"Page {page} retrieved")
    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"No results on page {page}, exiting loop")
        break

Page 0 retrieved
Page 1 retrieved
Page 2 retrieved
Page 3 retrieved
Page 4 retrieved
Page 5 retrieved
Page 6 retrieved
Page 7 retrieved
Page 8 retrieved
Page 9 retrieved
Page 10 retrieved
Page 11 retrieved
Page 12 retrieved
Page 13 retrieved
Page 14 retrieved
Page 15 retrieved
Page 16 retrieved
Page 17 retrieved
Page 18 retrieved
Page 19 retrieved


In [143]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df = json_normalize(reviews_list)
df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [123]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

# Some of the main headlines starts with 'Review' so added code to compare if the title start with Review and 
#added logic to extract the title between \u2018 and \u2019

def extract_title(headline):

    #Check if the headline starts with "Review" and extract the name accordingly in between quotes. Some data loss is expected when the 
    #movie title has Apostrophe
    if headline.startswith("Review"):
        return pd.Series([headline]).str.extract(r"\u2018(.*?)\u2019", expand=False).str.strip().values[0]
    # Else strip the name of the move in between quotes and ending in Review
    else:
        return pd.Series([headline]).str.extract(r"\u2018(.*?)\u2019 Review", expand=False).str.strip().values[0]

df['title'] = df['headline.main'].apply(extract_title)
df.tail(20)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
180,https://www.nytimes.com/2017/08/24/movies/tale...,"Twenty-five years after they split, two former...",The New York Times,subject: Movies;creative_works: Tales of an Im...,2017-08-24T16:22:43+0000,284,Review: ‘Tales of an Immoral Couple’: Love Mea...,None,None,Tales of an Immoral Couple,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None,Tales of an Immoral Couple
181,https://www.nytimes.com/2017/08/08/movies/afte...,"A couple break up, but remain together, frustr...",The New York Times,subject: Movies;creative_works: After Love (Mo...,2017-08-08T19:17:54+0000,274,Review: Staying Together ‘After Love’ and Regr...,None,None,After Love,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,After Love
182,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Last Face
183,https://www.nytimes.com/2017/07/25/movies/wome...,"In Ingrid Jungermann’s droll comedy, ex-lovers...",The New York Times,subject: Movies;creative_works: Women Who Kill...,2017-07-25T21:38:36+0000,411,"Review: She Hosts ‘Women Who Kill,’ and May Be...",None,None,"Killer Lines, Dangerous Dates",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,"Women Who Kill,"
184,https://www.nytimes.com/2017/07/13/movies/fals...,Isabelle Huppert and Louis Garrel star in a sc...,The New York Times,subject: Movies;creative_works: False Confessi...,2017-07-13T19:49:06+0000,527,"Review: With ‘False Confessions,’ the Play’s N...",None,None,Insolvent Gentleman Seeks Love,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"False Confessions,"
185,https://www.nytimes.com/2017/06/22/movies/my-j...,"In “My Journey Through French Cinema,” Mr. Tav...",The New York Times,subject: Documentary Films and Programs;person...,2017-06-22T21:14:56+0000,773,"Review: Those Movies, Himself — Bertrand Taver...",None,None,Reflecting on a Lifelong Love,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,NaN
186,https://www.nytimes.com/2017/06/22/movies/the-...,Kumail Nanjiani stars opposite Zoe Kazan in th...,The New York Times,subject: Movies;creative_works: The Big Sick (...,2017-06-22T09:00:31+0000,1073,"Review: In ‘The Big Sick,’ Comedy Is Hard, Lov...",None,None,Love and (Almost) Death,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"The Big Sick,"
187,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None,Lost in Paris
188,https://www.nytimes.com/2017/06/01/movies/vinc...,This combination crime drama and romance finds...,The New York Times,subject: Movies;creative_works: Vincent N Roxx...,2017-06-01T19:52:51+0000,287,"Review: In ‘Vincent N Roxxy,’ Love Is in the A...",None,None,Vincent N Roxxy,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,"Vincent N Roxxy,"
189,https://www.nytimes.com/2017/05/18/movies/ever...,Amandla Stenberg plays a teenager with an immu...,The New York Times,"subject: Movies;creative_works: Everything, Ev...",2017-05-18T21:02:53+0000,270,"Review: ‘Everything, Everything’ Pits Lov

In [122]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df['keywords'] = df['keywords'].apply(lambda x: extract_keywords(x) if isinstance(x, list) else x)
df.tail(20)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
180,https://www.nytimes.com/2017/08/24/movies/tale...,"Twenty-five years after they split, two former...",The New York Times,subject: Movies;creative_works: Tales of an Im...,2017-08-24T16:22:43+0000,284,Review: ‘Tales of an Immoral Couple’: Love Mea...,None,None,Tales of an Immoral Couple,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None,Tales of an Immoral Couple
181,https://www.nytimes.com/2017/08/08/movies/afte...,"A couple break up, but remain together, frustr...",The New York Times,subject: Movies;creative_works: After Love (Mo...,2017-08-08T19:17:54+0000,274,Review: Staying Together ‘After Love’ and Regr...,None,None,After Love,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,After Love
182,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Last Face
183,https://www.nytimes.com/2017/07/25/movies/wome...,"In Ingrid Jungermann’s droll comedy, ex-lovers...",The New York Times,subject: Movies;creative_works: Women Who Kill...,2017-07-25T21:38:36+0000,411,"Review: She Hosts ‘Women Who Kill,’ and May Be...",None,None,"Killer Lines, Dangerous Dates",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,"Women Who Kill,"
184,https://www.nytimes.com/2017/07/13/movies/fals...,Isabelle Huppert and Louis Garrel star in a sc...,The New York Times,subject: Movies;creative_works: False Confessi...,2017-07-13T19:49:06+0000,527,"Review: With ‘False Confessions,’ the Play’s N...",None,None,Insolvent Gentleman Seeks Love,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"False Confessions,"
185,https://www.nytimes.com/2017/06/22/movies/my-j...,"In “My Journey Through French Cinema,” Mr. Tav...",The New York Times,subject: Documentary Films and Programs;person...,2017-06-22T21:14:56+0000,773,"Review: Those Movies, Himself — Bertrand Taver...",None,None,Reflecting on a Lifelong Love,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,NaN
186,https://www.nytimes.com/2017/06/22/movies/the-...,Kumail Nanjiani stars opposite Zoe Kazan in th...,The New York Times,subject: Movies;creative_works: The Big Sick (...,2017-06-22T09:00:31+0000,1073,"Review: In ‘The Big Sick,’ Comedy Is Hard, Lov...",None,None,Love and (Almost) Death,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"The Big Sick,"
187,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None,Lost in Paris
188,https://www.nytimes.com/2017/06/01/movies/vinc...,This combination crime drama and romance finds...,The New York Times,subject: Movies;creative_works: Vincent N Roxx...,2017-06-01T19:52:51+0000,287,"Review: In ‘Vincent N Roxxy,’ Love Is in the A...",None,None,Vincent N Roxxy,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,"Vincent N Roxxy,"
189,https://www.nytimes.com/2017/05/18/movies/ever...,Amandla Stenberg plays a teenager with an immu...,The New York Times,"subject: Movies;creative_works: Everything, Ev...",2017-05-18T21:02:53+0000,270,"Review: ‘Everything, Everything’ Pits Lov

In [41]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = df['title'].to_list()
title_list

['The Attachment Diaries',
 'What',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journal for Jordan',
 'West Side

### Access The Movie Database API

In [102]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
url+tmdb_key_string

'https://api.themoviedb.org/3/search/movie?query=&api_key=008077e88128ba98fc7071127df91ef1'

In [114]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in df['title']:
     # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter > 0:
        time.sleep(10)
        #break
    #Add 1 to the request counter
    request_counter += 1
    #print("Movie Title",title)
    if pd.isnull(title):
        continue

    # Perform a "GET" request for The Movie Database
    url1 = url+title+tmdb_key_string
    #print(url1)
    response = requests.get(url1)

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        #print("Inside t ry",response)
        movie_data = response.json()['results'][0]
        #print("Movie Data",movie_data)

        # Get movie id
        movie_id = movie_data['id']


         # Make a request for a the full movie details
        movie_details_url=f"https://api.themoviedb.org/3/movie/{movie_id}?{tmdb_key_string}"
        #print("url", movie_details_url)
        
        # Execute "GET" request with url
        movie_details_response = requests.get(movie_details_url)
        movieid_data=movie_details_response.json()

        # Extract the genre names into a list
        genres = [genre['name'] for genre in movieid_data['genres']]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang['english_name'] for lang in movieid_data['spoken_languages']]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movieid_data['production_countries']]

        # Add the relevant data to a dictionary and
        movie_data = {
            'title': title,
            'original_title': movieid_data['original_title'],
            'budget': movieid_data['budget'],
            'genres': genres,
            'language': movieid_data['original_language'],
            'spoken_languages': spoken_languages,
            'homepage': movieid_data['homepage'],
            'overview': movieid_data['overview'],
            'popularity': movieid_data['popularity'],
            'runtime': movieid_data['runtime'],
            'revenue': movieid_data['revenue'],
            'release_date': movieid_data['release_date'],
            'vote_average': movieid_data['vote_average'],
            'vote_count': movieid_data['vote_count'],
            'production_countries': production_countries,
        }
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append(movie_data)

        print(f"Found: {title}")
    except:
        print(f"Not found: {title}")


Movie Title The Attachment Diaries
Found: The Attachment Diaries
Movie Title What
Found: What
Movie Title You Can Live Forever
Found: You Can Live Forever
Movie Title A Tourist’s Guide to Love
Found: A Tourist’s Guide to Love
Movie Title Other People’s Children
Found: Other People’s Children
Movie Title One True Loves
Found: One True Loves
Movie Title The Lost Weekend: A Love Story
Found: The Lost Weekend: A Love Story
Movie Title A Thousand and One
Found: A Thousand and One
Movie Title Your Place or Mine
Found: Your Place or Mine
Movie Title Love in the Time of Fentanyl
Found: Love in the Time of Fentanyl
Movie Title Pamela, a Love Story
Found: Pamela, a Love Story
Movie Title In From the Side
Found: In From the Side
Movie Title After Love
Found: After Love
Movie Title Alcarràs
Found: Alcarràs
Movie Title Nelly & Nadine
Found: Nelly & Nadine
Movie Title Lady Chatterley’s Lover
Found: Lady Chatterley’s Lover
Movie Title The Sound of Christmas
Found: The Sound of Christmas
Movie Title T

In [115]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "language": "es",
        "spoken_languages": [
            "Spanish"
        ],
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 5.486,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "production_countries": [
            "Argentina"
        ]
    },
   

In [116]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_df

,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",5.486,102,0,2021-10-07,3.000,4,[Argentina]
1,What,What...,0,"[Comedy, Drama]",en,[English],,"It's another ordinary day for Ricky, until he ...",7.471,0,0,2024-04-30,0.000,0,[Australia]
2,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,15055,2023-03-24,6.700,33,"[Canada, United States of America]"
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",24.422,96,0,2023-04-21,6.300,158,[United States of America]
4,Other People’s Children,Les Enfants des autres,0,"[Drama, Comedy]",fr,"[English, French]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.306,104,84178,2022-09-21,6.842,184,[France]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,The Other Half,The Other Half,0,"[Drama, Romance]",en,[English],http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,3.997,103,0,2016-12-02,6.300,24,[Canada]
194,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"[Turkish, English]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",15.217,111,413844,2017-03-10,6.173,246,"[Turkey, United States of America]"
195,Love & Taxes,Pared con pared,0,"[Romance, Comedy]",es,[Spanish],,She’s a young pianist preparing for an auditio...,176.179,98,0,2024-04-12,6.704,93,[Spain]
196,"Everybody Loves Somebody,",Everybody Loves Somebody,2000000,"[Romance, Comedy]",en,"[English, Spanish]",,"On the surface, Clara Barron seems to have it ...",10.332,102,0,2017-02-10,6.600,158,[Mexico]


### Merge and Clean the Data for Export

In [148]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge( tmdb_movies_df,df, on='title')
merged_df.tail(20)

,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
180,"Woodpeckers,",Carpinteros,0,[Drama],es,[Spanish],,Julián finds love and a reason for living in t...,1.734,108,...,"Review: ‘Woodpeckers,’ a Tale of Love and Agon...",None,None,Woodpeckers,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
181,I Do ... Until I Don,I Do... Until I Don't,0,[Comedy],en,[English],http://idountilidontmovie.com/,"In Vero Beach, Florida, a trio of couples at v...",8.627,103,...,"Review: In ‘I Do ... Until I Don’t,’ Love and ...",None,None,I Do ... Until I Don’t,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
182,Tales of an Immoral Couple,La vida inmoral de la pareja ideal,0,"[Comedy, Romance]",es,[Spanish],http://www.holamexicoff.com/lavidainmoralusa,Lucio and Martina used to share an indescribab...,10.413,91,...,Review: ‘Tales of an Immoral Couple’: Love Mea...,None,None,Tales of an Immoral Couple,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None
183,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",25.462,130,...,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
184,"Women Who Kill,",Women Who Kill,0,"[Comedy, Crime, Mystery, Romance, Thriller]",en,[English],http://www.wwkmovie.com/,Morgan and Jean work well together as true cri...,4.983,93,...,"Review: She Hosts ‘Women Who Kill,’ and May Be...",None,None,"Killer Lines, Dangerous Dates",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
185,"False Confessions,",False Confessions,0,[Documentary],en,[English],,Defence attorney Jane Fisher-Byrialsen is dete...,2.158,91,...,"Review: With ‘False Confessions,’ the Play’s N...",None,None,Insolvent Gentleman Seeks Love,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
186,"The Big Sick,",The Big Sick,5000000,"[Comedy, Drama, Romance]",en,"[English, Urdu]",http://thebigsickmovie.com,Pakistan-born comedian Kumail Nanjiani and gra...,25.824,120,...,"Review: In ‘The Big Sick,’ Comedy Is Hard, Lov...",None,None,Love and (Almost) Death,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
187,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,11.222,83,...,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
188,"Vincent N Roxxy,",Vincent N Roxxy,5000000,"[Drama, Crime, Thriller]",en,[English],,A small town loner and a rebellious punk rocke...,11.934,110,...,"Review: In ‘Vincent N Roxxy,’ Love Is in the A...",None,None,Vincent N Roxxy,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
189,"Everything, Everything","Everything, Everything",10000000,"[Drama, Romance]",en,[English],http://everythingeverythingmovie.com,A teenager who's lived a sheltered life becaus...,33.539,96,...,"Review: ‘Everything, Everything’ Pits Love Aga...",None,None,"Everything, Everything",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None


In [149]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing

columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', '\'',]

# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)
    
    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[col] = merged_df[col].str.replace(char, '')

# Display the fixed DataFrame
merged_df.head()

,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",5.486,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What,What...,0,"Comedy, Drama",en,English,,"It's another ordinary day for Ricky, until he ...",7.471,0,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",en,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",24.422,96,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,Other People’s Children,Les Enfants des autres,0,"Drama, Comedy",fr,"English, French",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.306,104,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [150]:
# Drop "byline.person" column
merged_df = merged_df.drop('byline.person', axis=1)
merged_df.head()

,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",5.486,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,What,What...,0,"Comedy, Drama",en,English,,"It's another ordinary day for Ricky, until he ...",7.471,0,...,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",en,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",24.422,96,...,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None
4,Other People’s Children,Les Enfants des autres,0,"Drama, Comedy",fr,"English, French",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.306,104,...,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None


In [133]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates('title').reset_index(drop=True)
merged_df.head()

,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",5.486,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,What,What...,0,"Comedy, Drama",en,English,,"It's another ordinary day for Ricky, until he ...",7.471,0,...,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",en,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",24.422,96,...,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None
4,Other People’s Children,Les Enfants des autres,0,"Drama, Comedy",fr,"English, French",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.306,104,...,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,The Other Half,The Other Half,0,"Drama, Romance",en,English,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,3.997,103,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None
193,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"Romance, Drama, War",en,"Turkish, English",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",15.217,111,...,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,None
194,Love & Taxes,Pared con pared,0,"Romance, Comedy",es,Spanish,,She’s a young pianist preparing for an auditio...,176.179,98,...,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,None
195,"Everybody Loves Somebody,",Everybody Loves Somebody,2000000,"Romance, Comedy",en,"English, Spanish",,"On the surface, Clara Barron seems to have it ...",10.332,102,...,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,None


In [134]:
# Export data to CSV without the index
merged_df.to_csv('output.csv', index=False)
